In [17]:
import pandas as pd

In [18]:
import numpy as np

In [19]:
dataset = pd.read_csv('NCAA_Tourney_2002_2019_update.csv')

In [20]:
dataset['team1_win'] = (dataset['team1_score']>dataset['team2_score']).astype(int)

In [21]:
## Establish Log5

In [22]:
### first calculate 
### E(W%) = points scored^11.5 / (points scored^11.5+points allowed^11.5)

In [23]:
dataset['exp_win1'] = (dataset['team1_adjoe']**11.5)/ ((dataset['team1_adjde']**11.5)+(dataset['team1_adjoe']**11.5))
dataset['exp_win2'] = (dataset['team2_adjoe']**11.5)/ ((dataset['team2_adjde']**11.5)+(dataset['team2_adjoe']**11.5))

In [24]:
dataset['team1_log5'] = (dataset['exp_win1'] - (dataset['exp_win1']*dataset['exp_win2']))/ (dataset['exp_win1']+dataset['exp_win2']-(2*dataset['exp_win1']*dataset['exp_win2']))

In [25]:
dataset[['team1_adjoe','team1_teamname','season','team2_teamname','team1_adjde','exp_win1','exp_win2','team1_log5']].head()

,team1_adjoe,team1_teamname,season,team2_teamname,team1_adjde,exp_win1,exp_win2,team1_log5
0,106.296,F Dickinson,2019,Prairie View,110.5500,0.389061,0.382990,0.506403
1,114.952,Belmont,2019,Temple,101.8860,0.800207,0.738939,0.585922
2,106.675,N Dakota St,2019,NC Central,109.8730,0.415885,0.208404,0.730051
3,109.886,Arizona St,2019,St John's,98.6844,0.774944,0.721882,0.570190
4,110.803,Minnesota,2019,Louisville,96.3335,0.833312,0.913200,0.322116


In [29]:
seed_diff = list(dataset['team1_seed'] - dataset['team2_seed'])
seed_compare = []
for i in seed_diff:
  if i >= 2:
    seed_compare.append(1)
  elif i <= 2:
    seed_compare.append(-1)
  else:
    seed_compare.append(0)

In [31]:
dataset['seed_diff'] = seed_compare

In [32]:
dataset[['game_id','exp_win1','exp_win2','team2_score','team1_score','team1_log5','team1_id','team2_id', 'seed_diff']].head()

,game_id,exp_win1,exp_win2,team2_score,team1_score,team1_log5,team1_id,team2_id,seed_diff
0,2019-1192.0-1341.0,0.389061,0.382990,76,82,0.506403,1192.0,1341.0,-1
1,2019-1125.0-1396.0,0.800207,0.738939,70,81,0.585922,1125.0,1396.0,-1
2,2019-1295.0-1300.0,0.415885,0.208404,74,78,0.730051,1295.0,1300.0,-1
3,2019-1113.0-1385.0,0.774944,0.721882,65,74,0.570190,1113.0,1385.0,-1
4,2019-1278.0-1257.0,0.833312,0.913200,76,86,0.322116,1278.0,1257.0,1


In [33]:
def distance(lat1, lon1, lat2, lon2):

    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    
    return distance

In [34]:
from math import sin, cos, sqrt, atan2, radians
dataset['dist1'] = dataset.apply(lambda row: distance(row['host_lat'], row['host_long'], row['team1_lat'], row['team1_long']), axis=1)
dataset['dist2'] = dataset.apply(lambda row: distance(row['host_lat'], row['host_long'], row['team2_lat'], row['team2_long']), axis=1)

In [35]:
dataset['diff_dist'] = dataset['dist1'] - dataset['dist2']